In [6]:
%reload_ext autoreload
%autoreload 2

import os
import sys

# Add the root folder to the module search path
root_folder = os.path.dirname(os.getcwd())
sys.path.append(root_folder)
print(os.getcwd())

/home/ubuntu/plotano/plotano


In [7]:
from plotano.db.qa_database import (
    QA_CSV_HEADER,
    QA_CSV_HEADER_ID, 
    QA_CSV_HEADER_QUESTION, 
    QA_CSV_HEADER_ANSWER, 
    QA_CSV_HEADER_VOTE_STATUS,
    QA_CSV_HEADER_TIMESTAMPS
)

print(QA_CSV_HEADER)

('ID', 'Question', 'Answer', 'Vote Status', 'Timestamp')


In [8]:
from plotano.db.qa_database import QuestionAnswerDatabase
db = QuestionAnswerDatabase(
    # db_file = "/Users/lingjiekong/Documents/github/cambioml/plotano/plotano/db/qa.db",
    debug=True)

Table contents after creating table:


In [9]:
db.create_table()

Table contents after creating table:


In [10]:
id = db.insert_question_answer(question="Who is Rachel?", answer="CEO of CambioML")
id = db.insert_question_answer(question="Who is Jared?", answer="CTO of CambioML")
id = db.insert_question_answer(question="Who is Kimi?", answer="Nobody of CambioML")

Table contents after inserting table:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.202246
Table contents after inserting table:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.202246
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.206972
Table contents after inserting table:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.202246
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.206972
ID: 3, Question: Who is Kimi?, Answer: Nobody of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.210210


In [11]:
db.update_vote_status(id=1, vote_status="up")
db.update_vote_status(id=2, vote_status="up")
db.update_vote_status(id=3, vote_status="down")

Table contents after updating table:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Vote Status: up, Timestamp: 2023-07-20 23:19:25.202246
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.206972
ID: 3, Question: Who is Kimi?, Answer: Nobody of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.210210
Table contents after updating table:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Vote Status: up, Timestamp: 2023-07-20 23:19:25.202246
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Vote Status: up, Timestamp: 2023-07-20 23:19:25.206972
ID: 3, Question: Who is Kimi?, Answer: Nobody of CambioML, Vote Status: n/a, Timestamp: 2023-07-20 23:19:25.210210
Table contents after updating table:
ID: 1, Question: Who is Rachel?, Answer: CEO of CambioML, Vote Status: up, Timestamp: 2023-07-20 23:19:25.202246
ID: 2, Question: Who is Jared?, Answer: CTO of CambioML, Vote Status: up, Timestamp: 2023-07-20

In [13]:
my_sql_data = db.retrieve_all_question_answers()
my_sql_data

[(1, 'Who is Rachel?', 'CEO of CambioML', 'up', '2023-07-20 23:19:25.202246'),
 (2, 'Who is Jared?', 'CTO of CambioML', 'up', '2023-07-20 23:19:25.206972'),
 (3,
  'Who is Kimi?',
  'Nobody of CambioML',
  'down',
  '2023-07-20 23:19:25.210210')]

In [14]:
from plotano.db.qa_database import QA_CSV_HEADER
header = QA_CSV_HEADER
print("header: {}".format(QA_CSV_HEADER))

header: ('ID', 'Question', 'Answer', 'Vote Status', 'Timestamp')


In [15]:
import csv

my_sql_data_path = 'my_sql_data.csv'
with open(my_sql_data_path, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(my_sql_data)

In [16]:
db.close_connection()

### Load my_data to RLHF

In [17]:
my_sql_data_path = 'my_sql_data.csv'


In [23]:
import pandas as pd
my_data_hf = pd.read_csv(my_sql_data_path)
my_data_hf

,ID,Question,Answer,Vote Status,Timestamp
0,1,Who is Rachel?,CEO of CambioML,up,2023-07-20 23:19:25.202246
1,2,Who is Jared?,CTO of CambioML,up,2023-07-20 23:19:25.206972
2,3,Who is Kimi?,Nobody of CambioML,down,2023-07-20 23:19:25.210210


In [24]:
common_header = [QA_CSV_HEADER_ID, QA_CSV_HEADER_QUESTION, QA_CSV_HEADER_ANSWER]

## filter only upvote data
my_data_hf_up = my_data_hf[my_data_hf[QA_CSV_HEADER_VOTE_STATUS] == 'up'][common_header]
my_data_hf_up

,ID,Question,Answer
0,1,Who is Rachel?,CEO of CambioML
1,2,Who is Jared?,CTO of CambioML


#### Add more data for training

In [25]:

stackoverflow_dataset = pd.read_parquet('/home/ubuntu/datasets/finetuned-train-00000-of-00020.parquet', engine='fastparquet', index=False)
my_stackoverflow_dataset = stackoverflow_dataset[['qid', 'question', 'response_j']]
my_stackoverflow_dataset.head()

,qid,question,response_j
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ..."
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...
2,141736,I've got a flight back to my own country with ...,12:45 pm is also 12:45 o'clock in internationa...
3,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...
4,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...


In [26]:
my_stackoverflow_dataset.columns = common_header
my_stackoverflow_dataset.head()

,ID,Question,Answer
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ..."
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...
2,141736,I've got a flight back to my own country with ...,12:45 pm is also 12:45 o'clock in internationa...
3,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...
4,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...


In [27]:
my_data_hf = pd.concat([my_data_hf, my_stackoverflow_dataset])
my_data_hf.head(10)

,ID,Question,Answer,Vote Status,Timestamp
0,1,Who is Rachel?,CEO of CambioML,up,2023-07-20 23:19:25.202246
1,2,Who is Jared?,CTO of CambioML,up,2023-07-20 23:19:25.206972
2,3,Who is Kimi?,Nobody of CambioML,down,2023-07-20 23:19:25.210210
0,12891264,I am using jQuery fileupload plugin and I want...,"Looking at the library code, seems all events ...",NaN,NaN
1,588860,When reading about the Big Bang you’ll commonl...,No. The local energy *density* is reduced as t...,NaN,NaN
2,141736,I've got a flight back to my own country with ...,12:45 pm is also 12:45 o'clock in internationa...,NaN,NaN
3,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...,NaN,NaN
4,1484683,I have a piece of code that works fine in the ...,I had this exact same problem. The only way I ...,NaN,NaN
5,54589610,Gradle sync is failing after I updated its ver...,Following steps helps me - \n\n1. Change the *...,NaN,NaN
6,58521105,The below code groups the result (a List of Cl...,You are selecting `InnerList` from non-grouped...,NaN,NaN


In [28]:
my_data_hf.head(100).to_csv(my_sql_data_path)

### Load to huggingface

In [31]:
from datasets import load_dataset
ds = load_dataset('csv', data_files=my_sql_data_path)

Found cached dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-077eb79c95e38daa/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 686.13it/s]


In [32]:
ds = ds['train'] # Convert DatasetDict to Dataset

ds = ds.train_test_split(test_size=0.1)
train_dataset = ds["train"]
test_dataset = ds["test"]

train_dataset, test_dataset


(Dataset({
     features: ['Unnamed: 0', 'ID', 'Question', 'Answer', 'Vote Status', 'Timestamp'],
     num_rows: 90
 }),
 Dataset({
     features: ['Unnamed: 0', 'ID', 'Question', 'Answer', 'Vote Status', 'Timestamp'],
     num_rows: 10
 }))

### Test RLHF code with DB

In [34]:
from plotano.rlhf.rlhf import RLHFConfig

config = RLHFConfig(
    base_model_path="meta-llama/Llama-2-7b-hf", ## "distilroberta-base", ##
    reward_model_path="databricks/dolly-v2-3b", 
    dataset_type="csv", ## "huggingface", ## 
    dataset_name='/home/ubuntu/plotano/plotano/my_sql_data.csv',
    max_steps=10,
    output_dir="../rlhf_tests/",
)

In [36]:
from plotano.rlhf.rlhf import SFT ##, RewardTrainer, RL

rlhf_step1_sft = SFT(config)

Found cached dataset csv (/home/ubuntu/.cache/huggingface/datasets/csv/default-077eb79c95e38daa/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 1/1 [00:00<00:00, 817.28it/s]
Loading cached split indices for dataset at /home/ubuntu/.cache/huggingface/datasets/csv/default-077eb79c95e38daa/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-6229905a46229d4c.arrow and /home/ubuntu/.cache/huggingface/datasets/csv/default-077eb79c95e38daa/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d/cache-e4b02ef5d2b42908.arrow
/opt/conda/envs/trl/lib/python3.10/site-packages/trl/trainer/utils.py:246: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictonnary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Size of the train set: 90.               Size of the validation set: 10


Loading checkpoint shards: 100%|██████████| 2/2 [00:51<00:00, 25.79s/it]
/opt/conda/envs/trl/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
Using pad_token, but it is not set yet.
/opt/conda/envs/trl/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/envs/trl/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:212: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [37]:
rlhf_step1_sft.train_and_save("../rlhf_tests/step1_07201638")

/opt/conda/envs/trl/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/envs/trl/lib/python3.10/site-packages/trl/trainer/utils.py:268: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/opt/conda/envs/trl/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:321: UserWarning: MatMul8bitLt: inputs will 

Step,Training Loss,Validation Loss
